# Which variable region or 16s primer set to use
- Liz downloaded the data from qiagen
- each sheet is a sample
- for each sample
    - Clean the data 
    - Save it

In [1]:
import pandas as pd
from collections import OrderedDict
import xlrd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
excel_file_fp = "all_sample_panel.xlsx"

In [3]:
xls = xlrd.open_workbook(excel_file_fp, on_demand=True)
sheet_names= xls.sheet_names()

In [4]:
len(sheet_names)

138

In [7]:
sheet_names[0:4]

['V1V2_HNO17_S8_R1 (paired) t...',
 'V1V2_HNO19_S9_R1 (paired) t...',
 'V1V2_HNO209_S14_R1 (paired)...',
 'V1V2_HNO223_S15_R1 (paired)...']

In [8]:
def clean_up_excel(excel_fp,sheet_names):
    sheet=sheet_names
    sample_name = sheet.split(' ')[0]
    df=pd.read_excel(excel_fp, sheet_name=sheet)
    df= df[["ID","Taxonomy","Combined Abundance"]]
    df.set_index('ID', drop=True, inplace=True)
    tax_dict = OrderedDict()
    for e,i in enumerate( df.Taxonomy):
        tmp_dict = OrderedDict()
        sx = i.split("; ")
        for u in sx:
            if 'D_0__' in u:
                tmp_dict['K']=u.replace('D_0__','')
            elif 'D_1__' in u:
                tmp_dict['P']=u.replace('D_1__','')
            elif 'D_2__' in u:
                tmp_dict['C']=u.replace('D_2__','')
            elif 'D_3__' in u:
                tmp_dict['O']=u.replace('D_3__','')
            elif 'D_4__' in u:
                tmp_dict['F']=u.replace('D_4__','')
            elif 'D_5__' in u:
                tmp_dict['G']=u.replace('D_5__','')
            else :
                tmp_dict['S']=u.replace('D_6__','')
        tax_dict[e] = tmp_dict
    dfx = pd.DataFrame( tax_dict ).T
    dfx=dfx[["K","P","C","O","F","G","S"]]
    dfx.index = df.index
    dfy =pd.concat([dfx, df],axis=1)
    return( [sample_name, dfy] )

In [9]:
def get_distribution_of_taxa(data, taxa='G'):
    samples = OrderedDict()
    for i in data.keys():
        x=data.get(i)[1][taxa].value_counts()
        x.name = i
        samples[i] =x 
    df =pd.concat( samples.values(), axis=1)
    df.fillna(0, inplace=True)
    return(df)

In [10]:
data = OrderedDict()
for i in sheet_names :
    n = i.split(' ')[0]
    data[n] = ( clean_up_excel(excel_file_fp, i) )

In [11]:
list(data.keys())[0:4]

['V1V2_HNO17_S8_R1',
 'V1V2_HNO19_S9_R1',
 'V1V2_HNO209_S14_R1',
 'V1V2_HNO223_S15_R1']

In [12]:
all_phylum = get_distribution_of_taxa(data,"P")

In [13]:
all_phylum.to_csv("all_samples_phylum.csv", index_label="taxa")

In [14]:
all_fam = get_distribution_of_taxa(data,"F")

In [15]:
all_fam.to_csv("all_samples_fam.csv", index_label="taxa")

In [16]:
all_genus = get_distribution_of_taxa(data,"G")

In [17]:
all_genus.to_csv("all_samples_genus.csv", index_label="taxa")

In [18]:
all_species = get_distribution_of_taxa(data,"S")

In [19]:
all_species.to_csv("all_samples_species.csv", index_label="taxa")